<a href="https://colab.research.google.com/github/umeshrawat/MachineLearning/blob/master/Predict_Volcanic_Eruption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


: 